In [ ]:
import json
import requests
from dateutil.parser import parse

import datetime as datetime

Using the Luchtmeetnet API: https://api-docs.luchtmeetnet.nl/#intro

## Over data

### NO2
De hoogste concentraties stikstofdioxide (NO2) komen voor tijdens de ochtend- en avondspits. Deze stof komt vrij door het (weg)verkeer, energieproductie en industrie. Daarnaast ontstaat NO2 uit een reactie tussen stikstofmonoxide en ozon. Het weer en de verkeersdrukte hebben grote invloed op de concentratie. De wettelijke norm is een jaargemiddelde van 40 (μg/m3)
* Slecht: > 80 (y tot 100)
* Matig: 30 - 80
* Goed: < 30

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### NO
De concentratie stikstofmonoxide (NO) is rond de ochtend- en avondspits hoger. Deze stof komt vrij bij het verbranden van brandstof door auto's, cv-installaties, de industrie en elektriciteitscentrales. Eenmaal in de lucht vindt er een chemisch proces plaats. stikstofmonoxide wordt dan omgezet in stikstofdioxide. Voor stikstofmonoxide bestaan geen wettelijke normen.
* Goed: < 50
* Matig: > 50 (y tot 100) 

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### O3
Ozon wordt niet rechtstreeks uitgestoten, maar wordt gevormd uit stikstofoxiden, vluchtige organische stoffen en koolmonoxide. De concentratie ozon (O3) is vooral afhankelijk van het weer. In heel Europa wordt de bevolking gewaarschuwd bij ozonconcentraties boven 180 (μg/m3). Een concentratie van 240 (μg/m3) is de Europese alarmdrempel.
* Goed: 0 - 40
* Matig: > 40 (y tot 100)

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### PM10
De dagelijkse concentratie fijn stof (PM10) is afhankelijk van het weer. In de steden zijn de concentraties overdag gemiddeld iets hoger dan 's nachts, vooral door de verkeersbijdrage. PM10 is een verzamelnaam voor zwevende, inhaleerbare deeltjes met een maximale doorsnede van 0,01 milimeter. De wettelijke norm is een jaargemiddelde van 40 (μg/m3). Daarnaast mag het daggemiddelde jaarlijks maximaal 35 keer hoger zijn dan 50 (μg/m3). 
* Goed: < 30
* Matig: 30 - 70
* Slecht: > 70

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### PM2.5
De dagelijkse concentratie fijn stof (PM2.5) is afhankelijk van het weer. In de steden zijn de concentraties overdag gemiddeld iets hoger dan ’s nachts, vooral door de verkeersbijdrage. PM2.5 is een verzamelnaam voor zwevende, inhaleerbare deeltjes met een maximale doorsnede van 0,0025 millimeter. De wettelijke norm is een jaargemiddelde van 25 (μg/m3). Doordat PM2.5 nog kleiner is dan PM10 kunnen deze deeltjes dieper doordringen in de longen en zijn ze schadelijker voor de gezondheid.

PM2.5 apparatuur is gevoelig voor condensvorming. Bij hoge buitentemperaturen is de kans op condensvorming groter. De meetapparatuur keurt een meting automatisch af wanneer condensvorming verwacht wordt. Volgens de automatische instellingen in de meetapparatuur wordt condensvorming verwacht wanneer het verschil tussen de buitenluchttemperatuur en temperatuur in het meetinstrument klein is. Bij hoge buitenluchttemperaturen is de kans groot dat het temperatuurverschil klein is en de metingen automatisch worden afgekeurd. Als gevolg hiervan ontbreken PM2.5 metingen soms als de buitentemperatuur hoog is. Indien later bij controle door de meetnetbeheerder blijkt dat een meting toch correct heeft plaatsgevonden worden metingen alsnog goedgekeurd.
* Goed: < 20
* Matig: 20 - 50
* Slecht: > 50

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

In [ ]:
# check: thresholds & good / mediocre / bad
# check: yesterday
# check: yesterday, past hour / week / month / season (?) / year

In [ ]:
response = requests.get(f"https://api.luchtmeetnet.nl/open_api/stations/NL10404/measurements?formula=PM25") 
result_json = response.json()

In [ ]:
d = response.json()['data'][0]

In [ ]:
class Metric:
    
    ub_bad = float('inf') # why not?
    
    def __repr__(self):
        return f"{self.formula}-object"
    
    def __init__(self, formula, ub_good=None, ub_med=None, ub_bad=None):    
        self.formula = formula
        
        if ub_good:
            self.ub_good = ub_good
        if ub_med:
            self.ub_med = ub_med
        if ub_bad: 
            self.ub_bad = ub_bad
            
    def judgement(self, value):
        if value <= self.ub_good:
            return "GOOD"
        elif value <= self.ub_med:
            return "MEDIOCRE"
        elif value <= self.ub_bad:
            return "BAD"
        else:
            return f"PANIC - {value}"
    

In [ ]:
# TODO add to metric
metrics = dict()
metrics['NO2'] = Metric('NO2', ub_good=30, ub_med=80, ub_bad=100)
metrics['NO'] = Metric('NO', ub_good=30, ub_med=100)
metrics['O3'] = Metric('O3', ub_good=40, ub_med=180)
metrics['PM10'] = Metric('PM10', ub_good=30, ub_med=70, ub_bad=100)
metrics['PM25'] = Metric('PM25', ub_good=20, ub_med=50, ub_bad=100)

In [ ]:
class API:
    url = f"https://api.luchtmeetnet.nl/open_api/measurements"
    
    def get(self, station=None, start=None, end=None, metric=None):
        request_url = self.url
        
        # blegh - query string params
        # TODO something with station
        qsp_to_add = ["station_number=NL10404"]
        
        if start and end:
            qsp_to_add.append(f"start={start}")
            qsp_to_add.append(f"end={end}")
        if metric:
            qsp_to_add.append(f"formula={metric.formula}")
            
        if len(qsp_to_add) > 0:
            stuff = "&".join(qsp_to_add)
            request_url += f"?{stuff}" 
        
        print(request_url)
        response = requests.get(request_url) 
        if response.status_code != 200:
            print(f"PANIC - {response.status_code} on {request_url}")
        else:
            response_json = response.json()
            
        if 'pagination' in response_json:
            current_page = response_json['pagination']['current_page']
            last_page = response_json['pagination']['last_page']
            if current_page != last_page:
                print("TODO - pagination needed")
        
        # TODO go through data, create measurements
        return response_json

In [ ]:
class Measurement:

    def __repr__(self):
        return f"{self.metric.formula}, {self.value}"
    
    def __init__(self, formula, value, timestamp):
        self.metric = metrics[formula]
        self.value = float(value) # better safe than sorry
        self.timestamp = parse(timestamp)
        
    def evaluate(self):
        judgement = self.metric.judgement(self.value)
        print(judgement)
        return True
    
    def compare(self, other_measurement):
        diff = self.value - other_measurement.value
        print(f"value diff {diff}")


In [ ]:
m = Measurement(d['formula'], d['value'], d['timestamp_measured'])

In [ ]:
one_day = datetime.timedelta(days=1)
yesterday = m.timestamp - one_day
time_string = yesterday.isoformat()
api = API() # dit doet dus precies niks, maar so be it
yesterdays_measurement = api.get(start=time_string, end=time_string, metric=m.metric)
ym = yesterdays_measurement['data'][0] #TODO
new_m = Measurement(ym['formula'], ym['value'], ym['timestamp_measured'])

In [ ]:
m.compare(new_m)

In [ ]:
new_m.value, m.value